In [61]:
%reload_ext autoreload
%autoreload 2

from mlp import MlpRun

In [62]:
LBP_PATH = "./data/lbp"
CLASS_NAMES = ["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]

In [63]:
batch_size = 128
learning_rate = 0.0001
dropout_rate = 0.2
hidden_sizes = [512, 256]
epochs = 150
output_size = 4  # Four classes

In [64]:
run = MlpRun(LBP_PATH, CLASS_NAMES, batch_size, hidden_sizes, output_size, dropout_rate, learning_rate)

Features dtype: torch.float32
Features shape: torch.Size([21165, 256])
Labels dtype: torch.int64
Labels shape: torch.Size([21165])


In [65]:
run.train(epochs)

[0 Val loss: 0.8374054747469285
New best model saved with accuracy: 66.87%
[1 Val loss: 0.7805933040731093
New best model saved with accuracy: 69.23%
[2 Val loss: 0.7481538267696605
New best model saved with accuracy: 69.47%
[3 Val loss: 0.722872116986443
New best model saved with accuracy: 71.55%
[4 Val loss: 0.7042934438761543
New best model saved with accuracy: 73.02%
[5 Val loss: 0.6868303032482371
New best model saved with accuracy: 73.20%
[6 Val loss: 0.6719904506907743
New best model saved with accuracy: 73.87%
[7 Val loss: 0.6631617265589097
[8 Val loss: 0.658650840029997
[9 Val loss: 0.6440713896470911
New best model saved with accuracy: 75.00%
[10 Val loss: 0.6414354092934552
New best model saved with accuracy: 75.09%
[11 Val loss: 0.6322532015688279
[12 Val loss: 0.6280340973068687
[13 Val loss: 0.6270003178540398
New best model saved with accuracy: 75.24%
[14 Val loss: 0.6308692097663879
[15 Val loss: 0.6210807035951054
New best model saved with accuracy: 75.61%
[16 Val los

accuracy,▁▂▄▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇███████▇
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▂▄▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇███████▇
recall,▁▂▄▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇███████▇
train_loss,█▇▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.79206
learning_rate,0.0001
precision,0.79206
recall,0.79206
train_loss,0.54546


In [66]:
run.test()